# Court Predict: NCAA Tournament Showdown

## Data Merging

### Libraries

In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

In [83]:
import os
import pandas as pd

file_directory = r'C:\\python practice\\machine learning\\NCAA_project\\data'
output_directory = r'C:\\python practice\\machine learning\\NCAA_project\\output'

DATA_PATH = 'C:\\python practice\\machine learning\\NCAA_project\\output\\'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

files = os.listdir(file_directory)

for file_name in files:
    if file_name.endswith('.csv'):
        df_name = file_name.split('.')[0]
        df = pd.read_csv(os.path.join(file_directory, file_name), encoding='latin1')
        df.to_csv(os.path.join(output_directory, f"{df_name}.csv"), index=False)

In [84]:
for filename in sorted(os.listdir(DATA_PATH)):
    print(filename)

2024_tourney_seeds.csv
Cities.csv
Conferences.csv
MConferenceTourneyGames.csv
MGameCities.csv
MMasseyOrdinals.csv
MNCAATourneyCompactResults.csv
MNCAATourneyDetailedResults.csv
MNCAATourneySeedRoundSlots.csv
MNCAATourneySeeds.csv
MNCAATourneySlots.csv
MRegularSeasonCompactResults.csv
MRegularSeasonDetailedResults.csv
MSeasons.csv
MSecondaryTourneyCompactResults.csv
MSecondaryTourneyTeams.csv
MTeamCoaches.csv
MTeamConferences.csv
MTeamSpellings.csv
MTeams.csv
SampleSubmission2023.csv
WGameCities.csv
WNCAATourneyCompactResults.csv
WNCAATourneyDetailedResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WRegularSeasonCompactResults.csv
WRegularSeasonDetailedResults.csv
WSeasons.csv
WTeamConferences.csv
WTeamSpellings.csv
WTeams.csv
df.csv
df_test.csv
fivethirtyeight_ncaa_forecasts_2023.csv
mens_elo_rating.csv
sample_submission.csv
womens_elo_rating.csv


In [85]:
df_seeds = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv"),
], ignore_index=True)
    
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [86]:
df_season_results = pd.concat([
    pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv"),
    pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv"),
], ignore_index=True)

df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)
df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,1985,20,1228,81,1328,64
1,1985,25,1106,77,1354,70
2,1985,25,1112,63,1223,56
3,1985,25,1165,70,1432,54
4,1985,25,1192,86,1447,74
...,...,...,...,...,...,...
316095,2024,100,3409,74,3427,70
316096,2024,100,3424,67,3161,64
316097,2024,100,3444,65,3296,47
316098,2024,100,3457,54,3347,51


**scoregap between W & L**

In [87]:
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,ScoreGap
0,1985,20,1228,81,1328,64,17
1,1985,25,1106,77,1354,70,7
2,1985,25,1112,63,1223,56,7
3,1985,25,1165,70,1432,54,16
4,1985,25,1192,86,1447,74,12


**each team's wins count**

In [88]:
# each team's # of wins 
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [89]:
num_win

,Season,TeamID,NumWins
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19
...,...,...,...
22096,2024,3474,3
22097,2024,3475,14
22098,2024,3476,3
22099,2024,3477,9


**each team's loss count**

In [90]:

num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

In [91]:
num_loss

,Season,TeamID,NumLosses
0,1985,1102,19
1,1985,1103,14
2,1985,1104,9
3,1985,1106,14
4,1985,1108,6
...,...,...,...
22125,2024,3474,18
22126,2024,3475,6
22127,2024,3476,20
22128,2024,3477,11


**points they scored more in average**

In [92]:
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})

In [93]:
gap_win


,Season,TeamID,GapWins
0,1985,1102,10.000000
1,1985,1103,7.555556
2,1985,1104,13.190476
3,1985,1106,9.500000
4,1985,1108,13.842105
...,...,...,...
22096,2024,3474,10.333333
22097,2024,3475,14.285714
22098,2024,3476,12.000000
22099,2024,3477,8.000000


**points they scored less in average**

In [94]:

gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})

In [95]:
gap_loss

,Season,TeamID,GapLosses
0,1985,1102,9.947368
1,1985,1103,9.857143
2,1985,1104,4.777778
3,1985,1106,13.285714
4,1985,1108,10.666667
...,...,...,...
22125,2024,3474,18.000000
22126,2024,3475,22.833333
22127,2024,3476,22.150000
22128,2024,3477,20.363636


In [96]:
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,ScoreGap
0,1985,20,1228,81,1328,64,17
1,1985,25,1106,77,1354,70,7
2,1985,25,1112,63,1223,56,7
3,1985,25,1165,70,1432,54,16
4,1985,25,1192,86,1447,74,12


In [97]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})

In [98]:
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [99]:
df_features_season_w

,Season,TeamID
0,1985,1102
1,1985,1103
2,1985,1104
3,1985,1106
4,1985,1108
...,...,...
22096,2024,3474
22097,2024,3475
22098,2024,3476
22099,2024,3477


In [100]:
df_features_season_l

,Season,TeamID
0,1985,1102
1,1985,1103
2,1985,1104
3,1985,1106
4,1985,1108
...,...,...
22125,2024,3474
22126,2024,3475
22127,2024,3476
22128,2024,3477


In [101]:
df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

In [102]:
df_features_season

,Season,TeamID
0,1985,1102
1,1985,1103
2,1985,1104
3,1985,1106
4,1985,1108
...,...,...
22145,2024,3474
22146,2024,3475
22147,2024,3476
22148,2024,3477


In [103]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')


In [104]:
df_features_season

,Season,TeamID,NumWins
0,1985,1102,5.0
1,1985,1103,9.0
2,1985,1104,21.0
3,1985,1106,10.0
4,1985,1108,19.0
...,...,...,...
22145,2024,3474,3.0
22146,2024,3475,14.0
22147,2024,3476,3.0
22148,2024,3477,9.0


In [105]:
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')


In [106]:
df_features_season

,Season,TeamID,NumWins,NumLosses
0,1985,1102,5.0,19.0
1,1985,1103,9.0,14.0
2,1985,1104,21.0,9.0
3,1985,1106,10.0,14.0
4,1985,1108,19.0,6.0
...,...,...,...,...
22145,2024,3474,3.0,18.0
22146,2024,3475,14.0,6.0
22147,2024,3476,3.0,20.0
22148,2024,3477,9.0,11.0


In [107]:
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')

In [108]:
df_features_season

,Season,TeamID,NumWins,NumLosses,GapWins
0,1985,1102,5.0,19.0,10.000000
1,1985,1103,9.0,14.0,7.555556
2,1985,1104,21.0,9.0,13.190476
3,1985,1106,10.0,14.0,9.500000
4,1985,1108,19.0,6.0,13.842105
...,...,...,...,...,...
22145,2024,3474,3.0,18.0,10.333333
22146,2024,3475,14.0,6.0,14.285714
22147,2024,3476,3.0,20.0,12.000000
22148,2024,3477,9.0,11.0,8.000000


In [109]:
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')

In [110]:
df_features_season

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses
0,1985,1102,5.0,19.0,10.000000,9.947368
1,1985,1103,9.0,14.0,7.555556,9.857143
2,1985,1104,21.0,9.0,13.190476,4.777778
3,1985,1106,10.0,14.0,9.500000,13.285714
4,1985,1108,19.0,6.0,13.842105,10.666667
...,...,...,...,...,...,...
22145,2024,3474,3.0,18.0,10.333333,18.000000
22146,2024,3475,14.0,6.0,14.285714,22.833333
22147,2024,3476,3.0,20.0,12.000000,22.150000
22148,2024,3477,9.0,11.0,8.000000,20.363636


In [111]:
df_features_season.fillna(0, inplace=True)  

In [112]:
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])

In [113]:
df_features_season

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses,WinRatio
0,1985,1102,5.0,19.0,10.000000,9.947368,0.208333
1,1985,1103,9.0,14.0,7.555556,9.857143,0.391304
2,1985,1104,21.0,9.0,13.190476,4.777778,0.700000
3,1985,1106,10.0,14.0,9.500000,13.285714,0.416667
4,1985,1108,19.0,6.0,13.842105,10.666667,0.760000
...,...,...,...,...,...,...,...
22145,2024,3474,3.0,18.0,10.333333,18.000000,0.142857
22146,2024,3475,14.0,6.0,14.285714,22.833333,0.700000
22147,2024,3476,3.0,20.0,12.000000,22.150000,0.130435
22148,2024,3477,9.0,11.0,8.000000,20.363636,0.450000


In [114]:
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

In [115]:
df_features_season

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses,WinRatio,GapAvg
0,1985,1102,5.0,19.0,10.000000,9.947368,0.208333,-5.791667
1,1985,1103,9.0,14.0,7.555556,9.857143,0.391304,-3.043478
2,1985,1104,21.0,9.0,13.190476,4.777778,0.700000,7.800000
3,1985,1106,10.0,14.0,9.500000,13.285714,0.416667,-3.791667
4,1985,1108,19.0,6.0,13.842105,10.666667,0.760000,7.960000
...,...,...,...,...,...,...,...,...
22145,2024,3474,3.0,18.0,10.333333,18.000000,0.142857,-13.952381
22146,2024,3475,14.0,6.0,14.285714,22.833333,0.700000,3.150000
22147,2024,3476,3.0,20.0,12.000000,22.150000,0.130435,-17.695652
22148,2024,3477,9.0,11.0,8.000000,20.363636,0.450000,-7.600000


In [116]:
df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)

In [117]:
df_features_season

,Season,TeamID,WinRatio,GapAvg
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000
...,...,...,...,...
22145,2024,3474,0.142857,-13.952381
22146,2024,3475,0.700000,3.150000
22147,2024,3476,0.130435,-17.695652
22148,2024,3477,0.450000,-7.600000


In [118]:
df_tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "WNCAATourneyCompactResults.csv"),
    pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv"),
], ignore_index=True)
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [119]:
df = df_tourney_results.copy()
df = df[df['Season'] >= 2016].reset_index(drop=True)


In [120]:
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,2016,137,3107,61,3196,59
1,2016,137,3113,74,3308,52
2,2016,137,3120,68,3385,57
3,2016,137,3124,89,3225,59
4,2016,137,3177,97,3241,67
...,...,...,...,...,...,...
912,2023,146,1274,88,1400,81
913,2023,146,1361,57,1166,56
914,2023,152,1163,72,1274,59
915,2023,152,1361,72,1194,71


In [121]:
df = pd.merge(
    df, 
    df_seeds,   
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})

In [122]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})

In [123]:
import re

def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

In [124]:
df['SeedW'] = df['SeedW'].apply(treat_seed)
df['SeedL'] = df['SeedL'].apply(treat_seed)

In [125]:
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL
0,2016,137,3107,61,3196,59,12,5
1,2016,137,3113,74,3308,52,2,15
2,2016,137,3120,68,3385,57,9,8
3,2016,137,3124,89,3225,59,1,16
4,2016,137,3177,97,3241,67,6,11
...,...,...,...,...,...,...,...,...
912,2023,146,1274,88,1400,81,5,2
913,2023,146,1361,57,1166,56,5,6
914,2023,152,1163,72,1274,59,4,5
915,2023,152,1361,72,1194,71,5,9


In [126]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
}).drop(columns='TeamID', axis=1)

In [127]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
}).drop(columns='TeamID', axis=1)

**df**

In [128]:
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250
...,...,...,...,...,...,...,...,...,...,...,...,...
912,2023,146,1274,88,1400,81,5,2,0.781250,7.281250,0.764706,10.441176
913,2023,146,1361,57,1166,56,5,6,0.812500,7.531250,0.636364,8.151515
914,2023,152,1163,72,1274,59,4,5,0.757576,13.424242,0.781250,7.281250
915,2023,152,1361,72,1194,71,5,9,0.812500,7.531250,0.906250,12.875000


In [129]:
def add_loosing_matches(df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
     }
    win_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('W')})
    win_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('L')})
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
    }
    lose_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('W')})
    lose_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('L')})
    
    win_df = df.copy()
    lose_df = df.copy()

    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], axis=0, sort=False)

In [130]:
df = add_loosing_matches(df)

In [131]:
df

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250
...,...,...,...,...,...,...,...,...,...,...,...,...
912,2023,146,1400,81,1274,88,2,5,0.764706,10.441176,0.781250,7.281250
913,2023,146,1166,56,1361,57,6,5,0.636364,8.151515,0.812500,7.531250
914,2023,152,1274,59,1163,72,5,4,0.781250,7.281250,0.757576,13.424242
915,2023,152,1194,71,1361,72,9,5,0.906250,12.875000,0.812500,7.531250


**DataFrame to CSV**

In [132]:
df.to_csv('C:\\python practice\\machine learning\\NCAA_project\\output\\df.csv', index=False)

In [133]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', # '538rating'
]

for col in cols_to_diff:
    df[col + 'Diff'] = df[col + 'A'] - df[col + 'B']

In [134]:
df_test = pd.read_csv("C:\\python practice\\machine learning\\NCAA_project\\output\\SampleSubmission2023.csv")

In [135]:
df_test

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5
...,...,...
130678,2023_3474_3476,0.5
130679,2023_3474_3477,0.5
130680,2023_3475_3476,0.5
130681,2023_3475_3477,0.5


In [136]:
%%time
def separate_id(df):
    df['Season']  = df['ID'].apply(lambda x: int(x.split('_')[0]))
    df['TeamIdA'] = df['ID'].apply(lambda x: int(x.split('_')[1]))
    df['TeamIdB'] = df['ID'].apply(lambda x: int(x.split('_')[2]))
    return df

df_test = separate_id(df_test)

CPU times: total: 125 ms
Wall time: 179 ms


In [137]:
%%time

df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'}).fillna('W01')

df_test = pd.merge(
    df_test, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'TeamIdB'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'}).fillna('W01')
df_test['SeedA'] = df_test['SeedA'].apply(treat_seed)
df_test['SeedB'] = df_test['SeedB'].apply(treat_seed)
df_test.head(30)

CPU times: total: 312 ms
Wall time: 424 ms


,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB
0,2023_1101_1102,0.5,2023,1101,1102,1,1
1,2023_1101_1103,0.5,2023,1101,1103,1,1
2,2023_1101_1104,0.5,2023,1101,1104,1,1
3,2023_1101_1105,0.5,2023,1101,1105,1,1
4,2023_1101_1106,0.5,2023,1101,1106,1,1
5,2023_1101_1107,0.5,2023,1101,1107,1,1
6,2023_1101_1108,0.5,2023,1101,1108,1,1
7,2023_1101_1110,0.5,2023,1101,1110,1,1
8,2023_1101_1111,0.5,2023,1101,1111,1,1
9,2023_1101_1112,0.5,2023,1101,1112,1,2


In [138]:
%%time

df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'GapWins': 'GapWinsA',
    'GapLosses': 'GapLossesA',
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA',
}).drop(columns='TeamID', axis=1)

df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'GapWins': 'GapWinsB',
    'GapLosses': 'GapLossesB',
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
}).drop(columns='TeamID', axis=1)

CPU times: total: 93.8 ms
Wall time: 107 ms


In [139]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', # '538rating'
]
for col in cols_to_diff:
    df_test[col + 'Diff'] = df_test[col + 'A'] - df_test[col + 'B']
    
# Compute Difference in Final Score (ScoreDiff) and whether or not the team won (WinA)
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

In [140]:
df_test

,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,SeedDiff,WinRatioDiff,GapAvgDiff
0,2023_1101_1102,0.5,2023,1101,1102,1,1,0.346154,-3.692308,0.437500,-0.125000,0,-0.091346,-3.567308
1,2023_1101_1103,0.5,2023,1101,1103,1,1,0.346154,-3.692308,0.645161,5.838710,0,-0.299007,-9.531017
2,2023_1101_1104,0.5,2023,1101,1104,1,1,0.346154,-3.692308,0.852941,13.676471,0,-0.506787,-17.368778
3,2023_1101_1105,0.5,2023,1101,1105,1,1,0.346154,-3.692308,0.400000,-3.066667,0,-0.053846,-0.625641
4,2023_1101_1106,0.5,2023,1101,1106,1,1,0.346154,-3.692308,0.233333,-10.033333,0,0.112821,6.341026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130678,2023_3474_3476,0.5,2023,3474,3476,1,1,0.192308,-14.615385,0.285714,-6.250000,0,-0.093407,-8.365385
130679,2023_3474_3477,0.5,2023,3474,3477,1,1,0.192308,-14.615385,0.406250,-5.000000,0,-0.213942,-9.615385
130680,2023_3475_3476,0.5,2023,3475,3476,1,1,0.346154,-3.461538,0.285714,-6.250000,0,0.060440,2.788462
130681,2023_3475_3477,0.5,2023,3475,3477,1,1,0.346154,-3.461538,0.406250,-5.000000,0,-0.060096,1.538462


**DataFrame to CSV**

In [141]:
df_test.to_csv('C:\\python practice\\machine learning\\NCAA_project\\output\\df_test.csv', index=False)

In [142]:
df_test.to_csv('df_test.csv', index=False)

## Model

In [143]:
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [144]:
target_feature = 'WinA'
avoid = ['ScoreDiff', 'Season', 'DayNum', 'A_Win']
features = [col for col in df.columns if col not in avoid]

In [145]:
features

['TeamIdA',
 'ScoreA',
 'TeamIdB',
 'ScoreB',
 'SeedA',
 'SeedB',
 'WinRatioA',
 'GapAvgA',
 'WinRatioB',
 'GapAvgB',
 'SeedDiff',
 'WinRatioDiff',
 'GapAvgDiff',
 'WinA']

In [146]:
features = ['TeamIdA',
            #'ScoreA',
            'TeamIdB',
            #'ScoreB',
            'SeedA',
            'SeedB',
            'WinRatioA',
            'GapAvgA',
            'WinRatioB',
            'GapAvgB',
            'SeedDiff',
            'WinRatioDiff',
            'GapAvgDiff']

In [147]:
%%time

def kfold_model(train_df, tst_df):
    cvs = []
    preds_test = []
    seasons = train_df['Season'].unique()
    
    for season in seasons[1:]:
        print(f'\nValidating on season {season}')
        X_train = train_df[train_df['Season'] < season][features].reset_index(drop = True).copy()
        X_val = train_df[train_df['Season'] == season][features].reset_index(drop = True).copy()
        
        y_train = train_df[train_df['Season'] < season][target_feature].reset_index(drop = True).copy()
        y_val = train_df[train_df['Season'] == season][target_feature].reset_index(drop = True).copy()
        
        tst_dataset = tst_df[features].copy()

        scaler = MinMaxScaler()
        scaler.fit(X_train)
        
        X_train = scaler.transform(X_train)        
        X_val = scaler.transform(X_val)
        tst_dataset = scaler.transform(tst_dataset)
        
        model = XGBClassifier(n_estimators = 1024, random_state = 85)
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0, early_stopping_rounds = 128)
        pred = model.predict_proba(X_val)[:, 1]
        
        pred_test = model.predict_proba(tst_dataset)[:, 1]
        preds_test.append(pred_test)
        
        loss = log_loss(y_val, pred)
        cvs.append(loss)
        
        print(f'\t -> Scored {loss:.4f}')
    print(f'\nLocal Cross Validation Score Is: {np.mean(cvs):.3f}', '\n')
    return preds_test

CPU times: total: 0 ns
Wall time: 0 ns


In [148]:
%%time
predictions = kfold_model(df, df_test)


Validating on season 2017
	 -> Scored 0.5614

Validating on season 2018


c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


	 -> Scored 0.6140

Validating on season 2019
	 -> Scored 0.4756

Validating on season 2021


c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


	 -> Scored 0.5723

Validating on season 2022
	 -> Scored 0.5601

Validating on season 2023


c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\python practice\machine learning\NCAA_project\NCAAenv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


	 -> Scored 0.5706

Local Cross Validation Score Is: 0.559 

CPU times: total: 7.59 s
Wall time: 837 ms


In [149]:
mean_predictions = np.mean(predictions, 0)

sub = df_test[['ID', 'Pred']].copy()
sub['Pred'] = mean_predictions

In [150]:
sub

,ID,Pred
0,2023_1101_1102,0.504024
1,2023_1101_1103,0.577009
2,2023_1101_1104,0.414708
3,2023_1101_1105,0.503512
4,2023_1101_1106,0.581595
...,...,...
130678,2023_3474_3476,0.525480
130679,2023_3474_3477,0.615856
130680,2023_3475_3476,0.511185
130681,2023_3475_3477,0.487244


In [151]:
seeds_2024=pd.read_csv(DATA_PATH+'2024_tourney_seeds.csv')
seeds_2024['Year']=2023
seeds_2024

,Tournament,Seed,TeamID,Year
0,M,W01,1345,2023
1,M,W02,1266,2023
2,M,W03,1243,2023
3,M,W04,1397,2023
4,M,W05,1181,2023
...,...,...,...,...
123,W,Z12,3405,2023
124,W,Z13,3387,2023
125,W,Z14,3241,2023
126,W,Z15,3436,2023


In [152]:
sub['Year'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub['Team1ID'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['Team2ID'] = sub['ID'].apply(lambda x: int(x.split('_')[2]))

In [153]:
sub = sub.merge(seeds_2024, left_on=['Year', 'Team1ID'], right_on=['Year', 'TeamID'], how='left')
sub.rename(columns={'Seed': 'Team1Seed'}, inplace=True)
sub.drop('TeamID', axis=1, inplace=True)

sub = sub.merge(seeds_2024, left_on=['Year', 'Team2ID'], right_on=['Year', 'TeamID'], how='left')
sub.rename(columns={'Seed': 'Team2Seed', 'Tournament_x':'Tournament'}, inplace=True)
sub.drop(['TeamID', 'Tournament_y'], axis=1, inplace=True)

In [154]:
sub=sub.dropna()
sub

,ID,Pred,Year,Team1ID,Team2ID,Tournament,Team1Seed,Team2Seed
1089,2023_1104_1112,0.623657,2023,1104,1112,M,X01,X02
1090,2023_1104_1113,0.787733,2023,1104,1113,M,X01,Z11
1093,2023_1104_1116,0.656792,2023,1104,1116,M,X01,Z08
1096,2023_1104_1120,0.746404,2023,1104,1120,M,X01,Y09
1099,2023_1104_1124,0.731454,2023,1104,1124,M,X01,X03
...,...,...,...,...,...,...,...,...
129989,2023_3437_3450,0.674034,2023,3437,3450,W,Y04,Y05
129991,2023_3437_3452,0.597004,2023,3437,3452,W,Y04,W10
130060,2023_3439_3450,0.728860,2023,3439,3450,W,Z01,Y05
130062,2023_3439_3452,0.835455,2023,3439,3452,W,Z01,W10


In [155]:
preds_w_seeds = sub.copy()
preds_w_seeds = preds_w_seeds.sort_values(by='Team1Seed')


def extract_seed_number(seed):
    return int(seed[1:])


def compare_seeds(seed1, seed2):
    seed1_num = extract_seed_number(seed1)
    seed2_num = extract_seed_number(seed2)
    seed1_prefix = seed1[0]
    seed2_prefix = seed2[0]

    if seed1_num < seed2_num:
        return -1
    elif seed1_num > seed2_num:
        return 1
    else:  
        if seed1_prefix < seed2_prefix:
            return -1
        elif seed1_prefix > seed2_prefix:
            return 1
        else:
            return 0
        
def determine_seeds(row):
    seed1 = row['Team1Seed']
    seed2 = row['Team2Seed']

    comparison = compare_seeds(seed1, seed2)
    if comparison < 0:
        row['HigherSeed'] = row['Team1ID']
        row['HigherSeedID'] = seed1
        row['LowerSeed'] = row['Team2ID']
        row['LowerSeedID'] = seed2
    elif comparison > 0:
        row['HigherSeed'] = row['Team2ID']
        row['HigherSeedID'] = seed2
        row['LowerSeed'] = row['Team1ID']
        row['LowerSeedID'] = seed1
    else:  # If seeds are equal
        if row['Team1ID'] < row['Team2ID']:
            row['HigherSeed'] = row['Team1ID']
            row['HigherSeedID'] = seed1
            row['LowerSeed'] = row['Team2ID']
            row['LowerSeedID'] = seed2
        else:
            row['HigherSeed'] = row['Team2ID']
            row['HigherSeedID'] = seed2
            row['LowerSeed'] = row['Team1ID']
            row['LowerSeedID'] = seed1

    return row            


**rearranging the data to create new ID and updating Pred column accordingly**

In [156]:
preds_w_seeds = preds_w_seeds.apply(determine_seeds, axis=1)

preds_w_seeds['ID'] = preds_w_seeds.apply(lambda x: f"{x['Year']}_{x['HigherSeed']}_{x['LowerSeed']}", axis=1)
preds_w_seeds['Pred'] = preds_w_seeds.apply(lambda x: 1 - x['Pred'] if x['Team1ID'] != x['HigherSeed'] else x['Pred'], axis=1)
preds_w_seeds = preds_w_seeds[['ID', 'Pred', 'Year', 'HigherSeedID', 'LowerSeedID', 'Tournament', 'HigherSeed', 'LowerSeed']]

preds_w_seeds['new_ID'] = preds_w_seeds['Tournament'] + '_' + preds_w_seeds['Year'].astype(str) + '_' + preds_w_seeds['HigherSeedID'] + '_' + preds_w_seeds['LowerSeedID']


**Function to simulate a single matchup**

In [157]:

def simulate_matchup(probability):
    return np.random.rand() < probability

**sample_submission as template for the output**

In [158]:

sample_submission = pd.read_csv(DATA_PATH+'sample_submission.csv')

submission = sample_submission.copy()
submission.rename(columns={'Team':'Winner'},inplace=True)

In [159]:
tourney_slots = pd.read_csv(DATA_PATH+'MNCAATourneySlots.csv')
tourney_slots = tourney_slots[tourney_slots['Season'] == 2023]

tourney_slots_M = tourney_slots.copy()
tourney_slots_M['Tournament'] = 'M'

tourney_slots_W = tourney_slots.copy()
tourney_slots_W['Tournament'] = 'W'

tourney_slots = pd.concat([tourney_slots_M, tourney_slots_W])

In [160]:
def create_round(slot):
    if slot[0] != 'R':
        return 0
    elif slot[0] == 'R':
        return int(slot[1])


In [161]:
tourney_slots['Round'] = tourney_slots['Slot'].apply(create_round)
tourney_slots = tourney_slots[tourney_slots['Round'] != 0]

In [162]:
preds_df = preds_w_seeds[['ID','new_ID','Year','HigherSeedID','LowerSeedID','Tournament','Pred']]


In [163]:
preds_df

,ID,new_ID,Year,HigherSeedID,LowerSeedID,Tournament,Pred
57517,2023_1345_1417,M_2023_W01_Z02,2023,W01,Z02,M,0.334823
57465,2023_1345_1364,M_2023_W01_X14,2023,W01,X14,M,0.834087
57559,2023_1345_1462,M_2023_W01_Y03,2023,W01,Y03,M,0.747506
57549,2023_1345_1452,M_2023_W01_X09,2023,W01,X09,M,0.661586
57537,2023_1345_1438,M_2023_W01_X04,2023,W01,X04,M,0.736388
...,...,...,...,...,...,...,...
80817,2023_3378_3151,W_2023_W08_Z16,2023,W08,Z16,W,0.852732
80815,2023_3376_3151,W_2023_W01_Z16,2023,W01,Z16,W,0.868368
80807,2023_3368_3151,W_2023_X15_Z16,2023,X15,Z16,W,0.490190
80844,2023_3405_3151,W_2023_Z12_Z16,2023,Z12,Z16,W,0.637515


**finding round 1 matchups**

In [164]:

tourney_2023 = tourney_slots.copy()


initial_matchups = tourney_2023[tourney_2023['Round'] == 1]

results = initial_matchups.merge(preds_df, left_on=['StrongSeed','WeakSeed','Season'], right_on=['HigherSeedID','LowerSeedID','Year'], how='left')

results.rename(columns={'Tournament_x':'Tournament'}, inplace=True)

results.drop('Tournament_y', axis=1, inplace=True)

In [165]:
results

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred
0,2023,R1W1,W01,W16,M,1,2023_3376_3313,W_2023_W01_W16,2023,W01,W16,0.868368
1,2023,R1W1,W01,W16,M,1,2023_1345_1192,M_2023_W01_W16,2023,W01,W16,0.860691
2,2023,R1W2,W02,W15,M,1,2023_1266_1436,M_2023_W02_W15,2023,W02,W15,0.867765
3,2023,R1W2,W02,W15,M,1,2023_3268_3221,W_2023_W02_W15,2023,W02,W15,0.869844
4,2023,R1W3,W03,W14,M,1,2023_1243_1286,M_2023_W03_W14,2023,W03,W14,0.813258
...,...,...,...,...,...,...,...,...,...,...,...,...
123,2023,R1Z6,Z06,Z11,W,1,2023_1395_1113,M_2023_Z06_Z11,2023,Z06,Z11,0.590676
124,2023,R1Z7,Z07,Z10,W,1,2023_3124_3104,W_2023_Z07_Z10,2023,Z07,Z10,0.617964
125,2023,R1Z7,Z07,Z10,W,1,2023_1321_1129,M_2023_Z07_Z10,2023,Z07,Z10,0.590428
126,2023,R1Z8,Z08,Z09,W,1,2023_1116_1228,M_2023_Z08_Z09,2023,Z08,Z09,0.451777


**ROUND 1**

In [166]:
results['outcome'] = results['Pred'].apply(simulate_matchup)

results['Team'] = results.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

results.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

results

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R1W1,W01,W16,M,1,2023_3376_3313,W_2023_W01_W16,2023,W01,W16,0.868368,True,W01
2,2023,R1W2,W02,W15,M,1,2023_1266_1436,M_2023_W02_W15,2023,W02,W15,0.867765,True,W02
4,2023,R1W3,W03,W14,M,1,2023_1243_1286,M_2023_W03_W14,2023,W03,W14,0.813258,True,W03
6,2023,R1W4,W04,W13,M,1,2023_1397_1418,M_2023_W04_W13,2023,W04,W13,0.818417,True,W04
8,2023,R1W5,W05,W12,M,1,2023_3328_3339,W_2023_W05_W12,2023,W05,W12,0.726487,False,W12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023,R1Z4,Z04,Z13,W,1,2023_1163_1233,M_2023_Z04_Z13,2023,Z04,Z13,0.768472,True,Z04
120,2023,R1Z5,Z05,Z12,W,1,2023_1388_1433,M_2023_Z05_Z12,2023,Z05,Z12,0.788839,False,Z12
122,2023,R1Z6,Z06,Z11,W,1,2023_3314_3345,W_2023_Z06_Z11,2023,Z06,Z11,0.694952,False,Z11
124,2023,R1Z7,Z07,Z10,W,1,2023_3124_3104,W_2023_Z07_Z10,2023,Z07,Z10,0.617964,False,Z10


In [167]:
round_1_results = results[['Tournament','Slot','Team','ID']]

round_1_results

,Tournament,Slot,Team,ID
0,M,R1W1,W01,2023_3376_3313
2,M,R1W2,W02,2023_1266_1436
4,M,R1W3,W03,2023_1243_1286
6,M,R1W4,W04,2023_1397_1418
8,M,R1W5,W12,2023_3328_3339
...,...,...,...,...
118,W,R1Z4,Z04,2023_1163_1233
120,W,R1Z5,Z12,2023_1388_1433
122,W,R1Z6,Z11,2023_3314_3345
124,W,R1Z7,Z10,2023_3124_3104


**ROUND 2**

In [168]:

round_2 = tourney_2023[tourney_2023['Round'] == 2]

In [169]:
round_2 = round_2.merge(round_1_results, left_on=['StrongSeed','Tournament'], right_on=['Slot','Tournament'], how='left')
round_2.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)

round_2.drop(['Slot_y'], axis=1, inplace=True)
round_2.drop_duplicates(subset=['Slot','Tournament'], inplace=True)


In [170]:
round_2 = round_2.merge(round_1_results, left_on=['WeakSeed','Tournament'], right_on=['Slot','Tournament'], how='left')
round_2.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)

round_2.drop(['Slot_y'], axis=1, inplace=True)

round_2.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

In [171]:
round_2['StrongSeed'] = round_2.apply(lambda x: x['Team_x'] if x['Team_x'] < x['Team_y'] else x['Team_y'], axis=1)
round_2['WeakSeed'] = round_2.apply(lambda x: x['Team_x'] if x['Team_x'] > x['Team_y'] else x['Team_y'], axis=1)

round_2.drop(['Team_x', 'Team_y'], axis=1, inplace=True)

In [172]:

round_2 = round_2.merge(preds_df, left_on=['StrongSeed','WeakSeed','Tournament'], right_on=['HigherSeedID','LowerSeedID','Tournament'], how='left')

round_2['outcome'] = round_2['Pred'].apply(simulate_matchup)

round_2['Team'] = round_2.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

round_2

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R2W1,W01,W08,M,2,2023_3376_3313,2023_1272_1194,2023_1345_1272,M_2023_W01_W08,2023,W01,W08,0.755750,False,W08
1,2023,R2W2,W02,W10,M,2,2023_1266_1436,2023_3112_3452,2023_1266_1425,M_2023_W02_W10,2023,W02,W10,0.766210,True,W02
2,2023,R2W3,W03,W06,M,2,2023_1243_1286,2023_1246_1344,2023_1243_1246,M_2023_W03_W06,2023,W03,W06,0.654138,True,W03
3,2023,R2W4,W04,W12,M,2,2023_1397_1418,2023_3328_3339,2023_1397_1331,M_2023_W04_W12,2023,W04,W12,0.766968,False,W12
4,2023,R2X1,X01,X08,M,2,2023_1104_1369,2023_1268_1452,2023_1104_1268,M_2023_X01_X08,2023,X01,X08,0.615834,True,X01
5,2023,R2X2,X02,X07,M,2,2023_3234_3368,2023_1281_1429,2023_1112_1281,M_2023_X02_X07,2023,X02,X07,0.581942,True,X02
6,2023,R2X3,X03,X11,M,2,2023_1124_1364,2023_1166_1301,2023_1124_1301,M_2023_X03_X11,2023,X03,X11,0.674836,False,X11
7,2023,R2X4,X04,X12,M,2,2023_1438_1202,2023_3257_3179,2023_1438_1158,M_2023_X04_X12,2023,X04,X12,0.608156,False,X12
8,2023,R2Y1,Y01,Y08,M,2,2023_1222_1297,2023_3329_3274,2023_1222_1234,M_2023_Y01_Y08,2023,Y01,Y08,0.736839,False,Y08
9,2023,R2Y2,Y02,Y07,M,2,2023_1400_1159,2023_3301_3343,2023_1400_1401,M_2023_Y02_Y07,2023,Y02,Y07,0.632500,False,Y07


In [173]:
round_2_results = round_2[['Tournament','Slot','Team','ID']]

round_2_results

,Tournament,Slot,Team,ID
0,M,R2W1,W08,2023_1345_1272
1,M,R2W2,W02,2023_1266_1425
2,M,R2W3,W03,2023_1243_1246
3,M,R2W4,W12,2023_1397_1331
4,M,R2X1,X01,2023_1104_1268
5,M,R2X2,X02,2023_1112_1281
6,M,R2X3,X11,2023_1124_1301
7,M,R2X4,X12,2023_1438_1158
8,M,R2Y1,Y08,2023_1222_1234
9,M,R2Y2,Y07,2023_1400_1401


**ROUND 3**

In [174]:
round_3 = tourney_2023[tourney_2023['Round'] == 3]

round_3 = round_3.merge(round_2_results, left_on=['StrongSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_3.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)
round_3.drop(['Slot_y'], axis=1, inplace=True)

round_3.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

round_3 = round_3.merge(round_2_results, left_on=['WeakSeed','Tournament'], right_on=['Slot','Tournament'], how='left')
round_3.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)

round_3.drop(['Slot_y'], axis=1, inplace=True)
round_3.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

In [175]:
round_3['StrongSeed'] = round_3.apply(lambda x: x['Team_x'] if x['Team_x'] < x['Team_y'] else x['Team_y'], axis=1)
round_3['WeakSeed'] = round_3.apply(lambda x: x['Team_x'] if x['Team_x'] > x['Team_y'] else x['Team_y'], axis=1)

round_3.drop(['Team_x', 'Team_y'], axis=1, inplace=True)

In [176]:
round_3 = round_3.merge(preds_df, left_on=['StrongSeed','WeakSeed','Tournament'], right_on=['HigherSeedID','LowerSeedID','Tournament'], how='left')

round_3['outcome'] = round_3['Pred'].apply(simulate_matchup)

round_3['Team'] = round_3.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

round_3

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R3W1,W08,W12,M,3,2023_1345_1272,2023_1397_1331,2023_1272_1331,M_2023_W08_W12,2023,W08,W12,0.361206,False,W12
1,2023,R3W2,W02,W03,M,3,2023_1266_1425,2023_1243_1246,2023_1266_1243,M_2023_W02_W03,2023,W02,W03,0.595065,False,W03
2,2023,R3X1,X01,X12,M,3,2023_1104_1268,2023_1438_1158,2023_1104_1158,M_2023_X01_X12,2023,X01,X12,0.704523,True,X01
3,2023,R3X2,X02,X11,M,3,2023_1112_1281,2023_1124_1301,2023_1112_1301,M_2023_X02_X11,2023,X02,X11,0.648220,False,X11
4,2023,R3Y1,Y05,Y08,M,3,2023_1222_1234,2023_1274_1245,2023_1274_1234,M_2023_Y05_Y08,2023,Y05,Y08,0.476784,False,Y08
5,2023,R3Y2,Y03,Y07,M,3,2023_1400_1401,2023_1462_1235,2023_1462_1401,M_2023_Y03_Y07,2023,Y03,Y07,0.616010,True,Y03
6,2023,R3Z1,Z05,Z09,M,3,2023_1242_1228,2023_1163_1388,2023_1388_1228,M_2023_Z05_Z09,2023,Z05,Z09,0.705491,True,Z05
7,2023,R3Z2,Z03,Z15,M,3,2023_1129_1421,2023_1211_1113,2023_1211_1421,M_2023_Z03_Z15,2023,Z03,Z15,0.865140,True,Z03
8,2023,R3W1,W05,W08,W,3,2023_3376_3378,2023_3417_3328,2023_3328_3378,W_2023_W05_W08,2023,W05,W08,0.519665,True,W05
9,2023,R3W2,W02,W11,W,3,2023_3268_3452,2023_3323_3228,2023_3268_3228,W_2023_W02_W11,2023,W02,W11,0.715184,True,W02


In [177]:
round_3_results = round_3[['Tournament','Slot','Team','ID']]
round_3_results

,Tournament,Slot,Team,ID
0,M,R3W1,W12,2023_1272_1331
1,M,R3W2,W03,2023_1266_1243
2,M,R3X1,X01,2023_1104_1158
3,M,R3X2,X11,2023_1112_1301
4,M,R3Y1,Y08,2023_1274_1234
5,M,R3Y2,Y03,2023_1462_1401
6,M,R3Z1,Z05,2023_1388_1228
7,M,R3Z2,Z03,2023_1211_1421
8,W,R3W1,W05,2023_3328_3378
9,W,R3W2,W02,2023_3268_3228


**ROUND 4**

In [178]:
round_4 = tourney_2023[tourney_2023['Round'] == 4]

round_4 = round_4.merge(round_3_results, left_on=['StrongSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_4.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)
round_4.drop(['Slot_y'], axis=1, inplace=True)

round_4.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

round_4 = round_4.merge(round_3_results, left_on=['WeakSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_4.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)
round_4.drop(['Slot_y'], axis=1, inplace=True)
    
round_4.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

In [179]:
round_4['StrongSeed'] = round_4.apply(lambda x: x['Team_x'] if x['Team_x'] < x['Team_y'] else x['Team_y'], axis=1)
round_4['WeakSeed'] = round_4.apply(lambda x: x['Team_x'] if x['Team_x'] > x['Team_y'] else x['Team_y'], axis=1)

#drop Team_x and Team_y
round_4.drop(['Team_x', 'Team_y'], axis=1, inplace=True)

In [180]:
round_4 = round_4.merge(preds_df, left_on=['StrongSeed','WeakSeed','Tournament'], right_on=['HigherSeedID','LowerSeedID','Tournament'], how='left')

round_4['outcome'] = round_4['Pred'].apply(simulate_matchup)

round_4['Team'] = round_4.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

round_4

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R4W1,W03,W12,M,4,2023_1272_1331,2023_1266_1243,2023_1243_1331,M_2023_W03_W12,2023,W03,W12,0.612112,True,W03
1,2023,R4X1,X01,X11,M,4,2023_1104_1158,2023_1112_1301,2023_1104_1301,M_2023_X01_X11,2023,X01,X11,0.727778,True,X01
2,2023,R4Y1,Y03,Y08,M,4,2023_1274_1234,2023_1462_1401,2023_1462_1234,M_2023_Y03_Y08,2023,Y03,Y08,0.469552,False,Y08
3,2023,R4Z1,Z03,Z05,M,4,2023_1388_1228,2023_1211_1421,2023_1211_1388,M_2023_Z03_Z05,2023,Z03,Z05,0.704952,False,Z05
4,2023,R4W1,W02,W05,W,4,2023_3328_3378,2023_3268_3228,2023_3268_3328,W_2023_W02_W05,2023,W02,W05,0.690933,True,W02
5,2023,R4X1,X01,X10,W,4,2023_3390_3179,2023_3208_3233,2023_3390_3208,W_2023_X01_X10,2023,X01,X10,0.834582,True,X01
6,2023,R4Y1,Y01,Y02,W,4,2023_3231_3437,2023_3428_3261,2023_3231_3428,W_2023_Y01_Y02,2023,Y01,Y02,0.581978,True,Y01
7,2023,R4Z1,Z03,Z04,W,4,2023_3397_3425,2023_3163_3326,2023_3326_3397,W_2023_Z03_Z04,2023,Z03,Z04,0.553670,True,Z03


In [181]:
round_4_results = round_4[['Tournament','Slot','Team','ID']]
round_4_results

,Tournament,Slot,Team,ID
0,M,R4W1,W03,2023_1243_1331
1,M,R4X1,X01,2023_1104_1301
2,M,R4Y1,Y08,2023_1462_1234
3,M,R4Z1,Z05,2023_1211_1388
4,W,R4W1,W02,2023_3268_3328
5,W,R4X1,X01,2023_3390_3208
6,W,R4Y1,Y01,2023_3231_3428
7,W,R4Z1,Z03,2023_3326_3397


**ROUND 5**

In [182]:
round_5 = tourney_2023[tourney_2023['Round'] == 5]

round_5 = round_5.merge(round_4_results, left_on=['StrongSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_5.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)

round_5.drop(['Slot_y'], axis=1, inplace=True)
round_5.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

round_5 = round_5.merge(round_4_results, left_on=['WeakSeed','Tournament'], right_on=['Slot','Tournament'], how='left')
#rename original columns
round_5.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)
#drop columns from merge
round_5.drop(['Slot_y'], axis=1, inplace=True)

#drop duplicates
round_5.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

In [183]:
def compare_seeds(seed1, seed2):
    seed_num1 = extract_seed_number(seed1)
    seed_num2 = extract_seed_number(seed2)

    if seed_num1 == seed_num2:
        return seed1 if seed1 < seed2 else seed2
    else:
        return seed1 if seed_num1 < seed_num2 else seed2

# Applying comparison logic to determine StrongSeed and WeakSeed
round_5['StrongSeed'] = round_5.apply(lambda x: compare_seeds(x['Team_x'], x['Team_y']), axis=1)
round_5['WeakSeed'] = round_5.apply(lambda x: x['Team_x'] if x['Team_x'] != x['StrongSeed'] else x['Team_y'], axis=1)

#drop Team_x and Team_y
round_5.drop(['Team_x', 'Team_y'], axis=1, inplace=True)
round_5

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y
0,2023,R5WX,X01,W03,M,5,2023_1243_1331,2023_1104_1301
1,2023,R5YZ,Z05,Y08,M,5,2023_1462_1234,2023_1211_1388
2,2023,R5WX,X01,W02,W,5,2023_3268_3328,2023_3390_3208
3,2023,R5YZ,Y01,Z03,W,5,2023_3231_3428,2023_3326_3397


In [184]:
round_5 = round_5.merge(preds_df, left_on=['StrongSeed','WeakSeed','Tournament'], right_on=['HigherSeedID','LowerSeedID','Tournament'], how='left')

round_5['outcome'] = round_5['Pred'].apply(simulate_matchup)

round_5['Team'] = round_5.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

round_5

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R5WX,X01,W03,M,5,2023_1243_1331,2023_1104_1301,2023_1104_1243,M_2023_X01_W03,2023,X01,W03,0.726948,True,X01
1,2023,R5YZ,Z05,Y08,M,5,2023_1462_1234,2023_1211_1388,2023_1388_1234,M_2023_Z05_Y08,2023,Z05,Y08,0.575779,True,Z05
2,2023,R5WX,X01,W02,W,5,2023_3268_3328,2023_3390_3208,2023_3390_3268,W_2023_X01_W02,2023,X01,W02,0.692680,True,X01
3,2023,R5YZ,Y01,Z03,W,5,2023_3231_3428,2023_3326_3397,2023_3231_3326,W_2023_Y01_Z03,2023,Y01,Z03,0.796102,True,Y01


In [185]:
round_5_results = round_5[['Tournament','Slot','Team','ID']]

round_5_results

,Tournament,Slot,Team,ID
0,M,R5WX,X01,2023_1104_1243
1,M,R5YZ,Z05,2023_1388_1234
2,W,R5WX,X01,2023_3390_3268
3,W,R5YZ,Y01,2023_3231_3326


**ROUND 6**

In [186]:
round_6 = tourney_2023[tourney_2023['Round'] == 6]

round_6 = round_6.merge(round_5_results, left_on=['StrongSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_6.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)
round_6.drop(['Slot_y'], axis=1, inplace=True)

round_6.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

round_6 = round_6.merge(round_5_results, left_on=['WeakSeed','Tournament'], right_on=['Slot','Tournament'], how='left')

round_6.rename(columns={'Slot_x':'Slot', 'Tournament_x':'Tournament'}, inplace=True)    
round_6.drop(['Slot_y'], axis=1, inplace=True)

#drop duplicates
round_6.drop_duplicates(subset=['Slot','Tournament'], inplace=True)

In [187]:
# Applying comparison logic to determine StrongSeed and WeakSeed
round_6['StrongSeed'] = round_6.apply(lambda x: compare_seeds(x['Team_x'], x['Team_y']), axis=1)
round_6['WeakSeed'] = round_6.apply(lambda x: x['Team_x'] if x['Team_x'] != x['StrongSeed'] else x['Team_y'], axis=1)

round_6.drop(['Team_x', 'Team_y'], axis=1, inplace=True)

In [188]:
round_6 = round_6.merge(preds_df, left_on=['StrongSeed','WeakSeed','Tournament'], right_on=['HigherSeedID','LowerSeedID','Tournament'], how='left')

round_6['outcome'] = round_6['Pred'].apply(simulate_matchup)

round_6['Team'] = round_6.apply(lambda x: x['HigherSeedID'] if x['outcome'] == True else x['LowerSeedID'], axis=1)

round_6

,Season,Slot,StrongSeed,WeakSeed,Tournament,Round,ID_x,ID_y,ID,new_ID,Year,HigherSeedID,LowerSeedID,Pred,outcome,Team
0,2023,R6CH,X01,Z05,M,6,2023_1104_1243,2023_1388_1234,2023_1104_1388,M_2023_X01_Z05,2023,X01,Z05,0.666044,False,Z05
1,2023,R6CH,X01,Y01,W,6,2023_3390_3268,2023_3231_3326,2023_3390_3231,W_2023_X01_Y01,2023,X01,Y01,0.532974,True,X01


In [189]:
round_6_results = round_6[['Tournament','Slot','Team','ID']]

round_6_results

,Tournament,Slot,Team,ID
0,M,R6CH,Z05,2023_1104_1388
1,W,R6CH,X01,2023_3390_3231


**ALL ROUNDS**

In [190]:
#concatentate rounds 1-6 results together
all_rounds = pd.concat([round_1_results, round_2_results, round_3_results, round_4_results, round_5_results, round_6_results])

all_rounds

,Tournament,Slot,Team,ID
0,M,R1W1,W01,2023_3376_3313
2,M,R1W2,W02,2023_1266_1436
4,M,R1W3,W03,2023_1243_1286
6,M,R1W4,W04,2023_1397_1418
8,M,R1W5,W12,2023_3328_3339
...,...,...,...,...
1,M,R5YZ,Z05,2023_1388_1234
2,W,R5WX,X01,2023_3390_3268
3,W,R5YZ,Y01,2023_3231_3326
0,M,R6CH,Z05,2023_1104_1388


In [191]:
print(all_rounds)

   Tournament  Slot Team              ID
0           M  R1W1  W01  2023_3376_3313
2           M  R1W2  W02  2023_1266_1436
4           M  R1W3  W03  2023_1243_1286
6           M  R1W4  W04  2023_1397_1418
8           M  R1W5  W12  2023_3328_3339
..        ...   ...  ...             ...
1           M  R5YZ  Z05  2023_1388_1234
2           W  R5WX  X01  2023_3390_3268
3           W  R5YZ  Y01  2023_3231_3326
0           M  R6CH  Z05  2023_1104_1388
1           W  R6CH  X01  2023_3390_3231

[126 rows x 4 columns]


**SAMPLE SUBMISSION**

In [192]:

df = sample_submission.copy()

df.drop('Team', axis=1, inplace=True)

submission = df.merge(all_rounds, left_on=['Tournament', 'Slot'], right_on=['Tournament', 'Slot'], how='left')
submission.drop('ID', axis=1, inplace=True)

submission

,RowId,Tournament,Bracket,Slot,Team
0,0,M,1,R1W1,W01
1,1,M,1,R1W8,W08
2,2,M,1,R1W5,W12
3,3,M,1,R1W4,W04
4,4,M,1,R1W6,W06
...,...,...,...,...,...
121,121,W,1,R4Y1,Y01
122,122,W,1,R4Z1,Z03
123,123,W,1,R5WX,X01
124,124,W,1,R5YZ,Y01


**return all null rows from submission**

In [193]:
submission[submission['Team'].isnull()]

,RowId,Tournament,Bracket,Slot,Team


**DataFrame to CSV**

In [194]:
submission.to_csv('submission.csv', index=False)

In [195]:
submission

,RowId,Tournament,Bracket,Slot,Team
0,0,M,1,R1W1,W01
1,1,M,1,R1W8,W08
2,2,M,1,R1W5,W12
3,3,M,1,R1W4,W04
4,4,M,1,R1W6,W06
...,...,...,...,...,...
121,121,W,1,R4Y1,Y01
122,122,W,1,R4Z1,Z03
123,123,W,1,R5WX,X01
124,124,W,1,R5YZ,Y01
